# Producer Utilities: Ableton stems renamer

[https://linktr.ee/fortytwoprod](https://linktr.ee/fortytwoprod)

This program helps you clean up the names of your Ableton stem files by removing prefixes and numbers from the file names in a folder and all its subfolders. Follow these simple steps to get started:

1. Set the `folder_path` variable to the path of the folder containing your Ableton stem files.
    - **On macOS**: Right-click on the folder and select "Get Info". Under the "General" section, highlight the text to the right of "Where" and press Command+C to copy the path. In this notebook, find the line that says `folder_path =` and replace the existing path with the one you copied. Make sure to add a `/` at the end and type the folder name afterwards.
    - **On Windows**: Right-click on the folder and select "Properties". Under the "General" tab, highlight the text to the right of "Location" and press Ctrl+C to copy the path. In this notebook, find the line that says `folder_path =` and replace the existing path with the one you copied. Make sure to add a `\` at the end and type the folder name afterwards.
2. Set the `prefix` variable to the prefix you want to remove from the file names.
3. Click "Run All" to execute the notebook.
    1. If a "Select kernel" dialog box appears at the top of the window, click on the first option.
    2. If a pop-up asks to install an ipykernel, select "Install".

In [3]:
# Enter your own prefix and folder path
prefix = "A"
folder_path = "/Users/rr/Desktop/STEMS"

In [4]:
import os
import re


def clean_file_name(file_name: str, prefix: str="A") -> str:
    """
    Remove the prefix and numbers leading the file name of an Ableton stem export.
    
    Args:
        filename (str): The original filename to be cleaned.
    
    Returns:
        str: The cleaned filename.
    """
    # Remove the prefix "A " if it exists
    cleaned = re.sub(rf'^{prefix}\s+', '', file_name)
    # Remove any numbers at the start of the filename
    cleaned = re.sub(r'^\d+', '', cleaned)
    # Remove any leading symbols like "-", "_" or whitespace
    cleaned = re.sub(r'^[-_ \s]+', '', cleaned)

    return cleaned


def rename_files_in_folder(folder_path: str, prefix: str="A") -> None:
    """
    Clean the file name of all audio files in a folder.

    Args:
        folder_path (str): The path to the folder containing the audio files.
    
    Returns:
        None
    """
    # Select audio files in folder
    old_file_names = os.listdir(folder_path)
    audio_exts = ('.wav', '.aif', '.aiff', '.mp3')
    old_file_names = [file_name for file_name in old_file_names if file_name.lower().endswith(audio_exts)]
    if not old_file_names:
        print("No audio files found.")
        return
    new_file_names = [clean_file_name(file_name, prefix) for file_name in old_file_names]

    # Check for renaming conflicts
    dont_rename = []
    for old, new in zip(old_file_names, new_file_names):
        if new_file_names.count(new) > 1:
            dont_rename.append(old)
    if dont_rename:
        print(f'These files will not be renamed as their new names will conflict:')
    for file_name in dont_rename:
        print(f'\t"{file_name} -> "{clean_file_name(file_name, prefix)}"')
    
    # Rename files
    for old in old_file_names:
        if old not in dont_rename:
            new = clean_file_name(old, prefix)
            if new != old:
                old_path = os.path.join(folder_path, old)
                new_path = os.path.join(folder_path, new)
                os.rename(old_path, new_path)
                print(f'Renamed "{old}" to "{new}"')


def rename_files(folder_path: str, prefix: str="A") -> None:
    """
    Clean the file name of all audio files in a folder and its subfolders.

    Args:
        folder_path (str): The path to the folder containing the audio files.
    
    Returns:
        None
    """
    for root, _, _ in os.walk(folder_path):
        print(f'Cleaning files in "{root}":')
        rename_files_in_folder(root, prefix)
        print("\n")

In [5]:
rename_files(folder_path, prefix)

Cleaning files in "/Users/rr/Desktop/STEMS":
These files will not be renamed as their new names will conflict:
	"A 1 drums.mp3 -> "drums.mp3"
	"drums.mp3 -> "drums.mp3"


Cleaning files in "/Users/rr/Desktop/STEMS/untitled folder":
These files will not be renamed as their new names will conflict:
	"A 1 drums.mp3 -> "drums.mp3"
	"A 4 drums copy.mp3 -> "drums copy.mp3"
	"A 2 drums.mp3 -> "drums.mp3"
	"A 1 drums copy.mp3 -> "drums copy.mp3"


Cleaning files in "/Users/rr/Desktop/STEMS/untitled folder 2":
No audio files found.


Cleaning files in "/Users/rr/Desktop/STEMS/WET":


